In [1]:
import os, math
from math import log, pow, sqrt

In [2]:
with open('query_list.txt', 'r') as fp:
    query_list = fp.read()
query_list = query_list.splitlines()

with open('doc_list.txt', 'r') as fp:
    doc_list = fp.read()
doc_list = doc_list.splitlines()

In [3]:
word_doc_num = {}
doc_word = {}
for doc in doc_list:
    with open('./docs/' + doc + '.txt', 'r') as fp:
        doc_content = fp.read()
    doc_content = doc_content.split(' ')
    doc_word[doc] = doc_content
    for word in doc_content:
        if word not in word_doc_num:
            word_doc_num[word] = {}
        if doc not in  word_doc_num[word]:
            word_doc_num[word][doc] = 0
        word_doc_num[word][doc] += 1
        
word_query_num = {}
query_word = {}
for query in query_list:
    with open('./queries/' + query + '.txt', 'r') as fp:
        query_content = fp.read()
    query_content = query_content.split(' ')
    query_word[query] = query_content
    for word in query_content:
        if word not in word_query_num:
            word_query_num[word] = {}
        if query not in  word_query_num[word]:
            word_query_num[word][query] = 0
        word_query_num[word][query] += 1

In [4]:
total_word = []
        
for word, num in word_query_num.items():
    total_word.append(word)
for word, num in word_doc_num.items():
    total_word.append(word)

total_word = list(dict.fromkeys(total_word))

In [5]:
max_df = 0
for a, b in word_doc_num.items():
    if len(b) > max_df:
        max_df = len(b)

In [11]:
doc_word_tfidf = {}
doc_length = {}
for doc, content in doc_word.items():
    doc_word_tfidf[doc] = {}
    doc_len_2 = 0
    for word in content:
        if word not in doc_word_tfidf[doc]:
            # 0.70924
            # tf = 1 + log(word_doc_num[word][doc])
            # idf = log(1 + max_df / len(word_doc_num[word]))
            # 0.71178
            tf = log(word_doc_num[word][doc] + 1)
            idf = log(max_df / len(word_doc_num[word]))
            
            doc_word_tfidf[doc][word] = tf * idf
            doc_len_2 += pow(tf * idf, 2)
    doc_length[doc] = sqrt(doc_len_2)

In [12]:
query_word_tfidf = {}
query_length = {}
for query, content in query_word.items():
    query_word_tfidf[query] = {}
    query_len_2 = 0
    for word in content:
        if word not in query_word_tfidf[query]:
            # 0.70924
            # tf = 1 + log(word_query_num[word][query])
            # idf = log(1 + max_df / len(word_doc_num[word])) if word in word_doc_num else 0
            # 0.71178
            tf = log(word_query_num[word][query] + 1)
            idf = log(max_df / len(word_doc_num[word])) if word in word_doc_num else 0
            
            query_word_tfidf[query][word] = tf * idf
            query_len_2 += pow(tf * idf, 2)
    query_length[query] = sqrt(query_len_2)

In [13]:
result = {}
for query in query_list:
    for doc in doc_list:
        cossim = 0
        for word, tfidf in query_word_tfidf[query].items():
            cossim = cossim + (tfidf * doc_word_tfidf[doc][word]) if word in doc_word_tfidf[doc] else cossim
        if query not in result:
            result[query] = {}
        result[query][doc] = cossim / (doc_length[doc] * query_length[query]) if doc_length[doc] * query_length[query] != 0 else 0

In [15]:
f = open('test.txt', 'w+')
f.write('Query,RetrievedDocuments\n')

for query in query_list:
    print_ans = query + ','
    temp_result = sorted(result[query].items(), key=lambda x:x[1],reverse=True)
    for i in temp_result:
        print_ans += i[0] + ' '
    print_ans = print_ans[:-1] + '\n'
    f.write(print_ans)

f.close()

In [14]:
sorted(result['301'].items(), key=lambda x:x[1],reverse=True)

[('FBIS3-23986', 0.18594257509912146),
 ('FT941-3237', 0.1586229900347684),
 ('FBIS3-55219', 0.15746284359932153),
 ('FBIS3-19199', 0.15107469712344845),
 ('FBIS3-38070', 0.1506350618648429),
 ('FBIS3-26415', 0.14875273467707073),
 ('LA090489-0059', 0.14830277735142708),
 ('FBIS3-41247', 0.14778700710995077),
 ('FBIS3-19646', 0.14417509369529707),
 ('FBIS3-21961', 0.14417509369529707),
 ('FBIS4-43965', 0.140455179313319),
 ('FT943-16238', 0.1371526545072929),
 ('FBIS3-15586', 0.13142277723719578),
 ('FBIS3-58028', 0.12624518065282048),
 ('FBIS3-58058', 0.12624518065282048),
 ('FBIS4-43801', 0.1262391717168662),
 ('FBIS4-45477', 0.12510300136989153),
 ('FBIS4-7811', 0.1219651241593608),
 ('FBIS4-21321', 0.1176102964187369),
 ('FBIS4-8957', 0.11684760347463789),
 ('FT911-2686', 0.11485091352468367),
 ('FBIS3-21770', 0.11414168624760286),
 ('FBIS3-38878', 0.11252441497023341),
 ('FBIS3-60984', 0.11200926330302304),
 ('LA032790-0103', 0.1114199115439807),
 ('FBIS3-58055', 0.107620472216578